## Import required libraries

In [1]:
import pandas as pd
import numpy as np
import os

import datetime
from datetime import date, timedelta

#from tqdm.notebook import tqdm
from tqdm import tqdm_notebook

from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error


import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

#pd.set_option('precision', 3)

## Earth continents

In [2]:
continents = {}
continents['europe']=['Portugal', 'Italy', 'Germany', 'Finland', 'Estonia', 'Hungary', 
                      'Spain', 'Slovakia', 'Ireland', 'Iceland', 'France',  'Norway', 
                      'Lithuania', 'Turkey', 'Switzerland', 'Belgium', 'Denmark',  
                      'Poland', 'Austria', 'Sweden', 'CzechRepublic', 'Netherlands',
                      'UnitedKingdom',  'Latvia', 'Greece', 'Luxemborg']
continents['north_america']= 'USA Canada'.split()
continents['south_america']=['Ecuador', 'Panama', 'Peru',  'Uruguay', 'CostaRica',
                            'Mexico', 'Argentina', 'ElSalvador', 'Chile', 'Brazil', 
                            'Honduras', 'Guatemala','Bolivia', 'Columbia','DominicanRepublic', 'Paraguay']
continents['oceania']='Australia NewZealand'.split()
continents['asia']= ['Indonesia', 'Malaysia', 'Philippines', 'Taiwan', 'Japan', 'Singapore', 'HongKong']

In [3]:
continents['all']=[]
continents['all'].extend(continents['asia'])
continents['all'].extend(continents['oceania'])
continents['all'].extend(continents['south_america'])
continents['all'].extend(continents['north_america'])
continents['all'].extend(continents['europe'])

## Danceability clusters

In [4]:
dance_clust = {}
dance_clust['1']=[ "Portugal", "Italy",    "Finland",  "Estonia",  "Hungary", 
                  "Slovakia", "Ireland",  "Iceland",  "Brazil",   "Canada",  
                  "Belgium",  "Norway",   "Lithuania","New Zealand"    "Turkey",  
                  "USA", "Switzerland"    "Denmark",  "Poland",   "Austria", 
                  "Sweden",   "Czech Republic" "Netherlands",    "UK", 
                  "Australia","Latvia"]

dance_clust['2']= ["Ecuador",     "Panama",      "Germany",     "Spain",      
                   "Peru", "Uruguay",     "Costa Rica",  "Mexico",     
                   "France",      "Argentina",   "El Salvador", "Chile",      
                   "Honduras",    "Guatemala",   "Bolivia",     "Colombia",   
                   "Dominican Republic" "Paraguay",    "Greece",]

dance_clust['3']=[ "Indonesia" ,  "Malaysia" ,   "Philippines", "Taiwan",      "Japan",       "Singapore",   "HongKong"]

## Valence clusters

In [5]:
valence_clust = {}
valence_clust['1']=[ "Portugal",  "Italy","Germany",   "Finland",   "Estonia",  
                    "Hungary",   "Slovakia",  "Ireland",   "Iceland",   "Canada",   
                    "Belgium",   "France",    "Lithuania", "New Zealand"    "Turkey",   
                    "Japan","Switzerland"    "Denmark",   "Poland",    "Austria",  
                    "Sweden",    "Czech Republic", "Netherlands",    "UK",      
                    "Australia", "Latvia",    "Greece"   ]

valence_clust['2']= ["Indonesia","Malaysia", "Norway","Philippines" "Taiwan","USA","Singapore",  "HongKong" ]

valence_clust['3']=["Ecuador", "Panama",  "Spain",   "Peru",   
                    "Brazil",  "Uruguay", "Costa Rica" ,        "Mexico", 
                    "Argentina"  ,        "El Salvador" ,       "Chile",   "Honduras",
                    "Guatemala" ,         "Bolivia", "Colombia","Dominican Republic"
                    "Paraguay"]

## Energy clusters

In [6]:
energy_clust = {}
energy_clust['1']=["Indonesia" ,  "Iceland" ,    "Malaysia",    "Philippines",
                   "Taiwan",      "New Zealand", "USA",        "Singapore","HongKong"   ]

energy_clust['2']= ["Portugal" ,      "Estonia" ,       "Slovakia"  ,     "Ireland" ,       "Canada" ,       
                    "Belgium",        "Mexico" ,        "France"    ,     "Norway" ,        "Lithuania",
                    "Turkey"  ,       "Switzerland"    "Denmark" ,       "Sweden"  ,       "Czech Republic",
                    "Netherlands" ,   "UK"       ,      "World"   ,       "Australia" ,     "Latvia"   ,     
                    "Greece" ]

energy_clust['3']=["Ecuador",    "Panama",     "Italy",      "Germany",                
                   "Finland",    "Hungary",    "Spain",      "Peru",
                   "Brazil",     "Uruguay",    "Costa Rica", "Argentina", 
                   "El Salvador","Chile",      "Japan", "Honduras",  
                   "Guatemala",  "Bolivia",    "Poland",     "Austria",   
                   "Colombia",   "Dominican Republic", "Paraguay"]

In [7]:
resample_freq='W'
train_clust= 'en_3'
eval_clust= 'en_1'

train_countries = energy_clust['3']
eval_countries= energy_clust['2']

In [8]:
ranking_features_df= pd.read_csv(os.path.join('generated_data', 'ranking_features.csv'), index_col=0, parse_dates=['Date', 'release_date'])
ranking_features_df.head()

,Position,Track Name,Artist,Streams,URL,Date,Region,song_id,release_date,danceability,valence,energy,id
0,1,Échame La Culpa,"Luis Fonsi, Demi Lovato",26459,1zsG4eaZmkA1dvjDDsAGLK,2018-01-01,Ecuador,1zsG4eaZmkA1dvjDDsAGLK,2017-11-17,0.724,0.64,0.895,1zsG4eaZmkA1dvjDDsAGLK
1,2,Échame La Culpa,"Luis Fonsi, Demi Lovato",24103,1zsG4eaZmkA1dvjDDsAGLK,2018-01-02,Ecuador,1zsG4eaZmkA1dvjDDsAGLK,2017-11-17,0.724,0.64,0.895,1zsG4eaZmkA1dvjDDsAGLK
2,3,Échame La Culpa,"Luis Fonsi, Demi Lovato",24702,1zsG4eaZmkA1dvjDDsAGLK,2018-01-03,Ecuador,1zsG4eaZmkA1dvjDDsAGLK,2017-11-17,0.724,0.64,0.895,1zsG4eaZmkA1dvjDDsAGLK
3,3,Échame La Culpa,"Luis Fonsi, Demi Lovato",24584,1zsG4eaZmkA1dvjDDsAGLK,2018-01-04,Ecuador,1zsG4eaZmkA1dvjDDsAGLK,2017-11-17,0.724,0.64,0.895,1zsG4eaZmkA1dvjDDsAGLK
4,3,Échame La Culpa,"Luis Fonsi, Demi Lovato",25531,1zsG4eaZmkA1dvjDDsAGLK,2018-01-05,Ecuador,1zsG4eaZmkA1dvjDDsAGLK,2017-11-17,0.724,0.64,0.895,1zsG4eaZmkA1dvjDDsAGLK


In [9]:
ranking_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18749500 entries, 0 to 18749499
Data columns (total 13 columns):
 #   Column        Dtype         
---  ------        -----         
 0   Position      int64         
 1   Track Name    object        
 2   Artist        object        
 3   Streams       int64         
 4   URL           object        
 5   Date          datetime64[ns]
 6   Region        object        
 7   song_id       object        
 8   release_date  datetime64[ns]
 9   danceability  float64       
 10  valence       float64       
 11  energy        float64       
 12  id            object        
dtypes: datetime64[ns](2), float64(3), int64(2), object(6)
memory usage: 2.0+ GB


## Read weather data

In [10]:
weather_df= pd.read_csv(os.path.join('generated_data','all_weather.csv'), header=[0,1], index_col=0)
weather_df.index= pd.to_datetime(weather_df.index)
weather_df[('country','World')]= weather_df.mean(axis=1)
weather_df.head()

country                                                       \
           Netherlands Philippines Singapore       Peru   Denmark     Italy   
date                                                                          
2017-01-01    1.729167   27.250000   27.4125  22.450000  5.645833  5.000000   
2017-01-02    3.208333   27.208333   26.5875  21.314286  0.875000  8.191489   
2017-01-03    5.375000   28.333333   26.3375  21.157143  3.306122  9.791667   
2017-01-04    6.145833   27.750000   26.7500  22.150000  1.958333  5.354167   
2017-01-05    0.708333   26.708333   27.9000  23.025000 -5.729167  5.729167   

                                                      ...            \
                Chile Lithuania  Slovakia  Indonesia  ...   Hungary   
date                                                  ...             
2017-01-01  25.357143    5.0750 -6.052632  31.444444  ... -6.711538   
2017-01-02  26.285714    1.1375 -4.540000  31.407407  ... -5.560000   
2017-01-03  23.785714   -1.9875  1.160000  30.115385  ... -0.354167   
2017-01-04  23.454545   -2.9125  3.345455  27.448276  ...  0.218182   
2017-01-05  26.285714  -13.5375 -1.041667  28.592593  ... -0.788462   

                                                                        \
            Argentina Poland  Bolivia  Guatemala   Honduras     Taiwan   
date                                                                     
2017-01-01  30.304348    NaN   9.2250        NaN  20.842857  22.642857   
2017-01-02  25.375000    NaN   9.7200  22.600000  20.612500  21.422222   
2017-01-03  25.533333    NaN  10.6250  19.533333  19.400000  20.509434   
2017-01-04  26.291667    NaN   8.7375  12.800000  20.325000  23.380952   
2017-01-05  24.826087    NaN   7.7250        NaN  19.950000  22.395833   

                                                
              Estonia CzechRepublic      World  
date                                            
2017-01-01   3.162500     -6.296296  11.250720  
2017-01-02  -2.175000     -3.458333  10.911742  
2017-01-03  -4.279167     -0.321429  11.447292  
2017-01-04  -8.895833      0.705882  11.365506  
2017-01-05 -12.326087     -3.437500   9.124424  

[5 rows x 54 columns]

In [11]:
train_weather= weather_df.iloc[:, weather_df.columns.get_level_values(1).isin(train_countries)].mean(axis=1)
if resample_freq!= 'D':
    train_weather = train_weather.resample(resample_freq).mean()
train_weather.head()

date
2017-01-01    13.724146
2017-01-08    13.026443
2017-01-15    13.737897
2017-01-22    13.667816
2017-01-29    13.370844
Freq: W-SUN, dtype: float64

In [12]:
eval_weather= weather_df.iloc[:, weather_df.columns.get_level_values(1).isin(eval_countries)].mean(axis=1)
if resample_freq!= 'D':
    eval_weather = eval_weather.resample(resample_freq).mean()
eval_weather.head()

date
2017-01-01    4.398201
2017-01-08    1.884014
2017-01-15    4.045547
2017-01-22    3.638521
2017-01-29    4.218135
Freq: W-SUN, dtype: float64

## Merge all data

In [13]:
def generate_target_multivariate_timeseries(songs_df, country_lst, resample=None):
    
    country_df = ranking_features_df[ranking_features_df['Region'].isin(country_lst)]
    daily_country_df = country_df.drop(columns='Position Streams'.split()).groupby('Date').mean()
 
    if resample != 'D':
        daily_country_df = daily_country_df.resample(resample).mean()
    
    return daily_country_df

In [14]:

train_ts = generate_target_multivariate_timeseries(ranking_features_df, 
                                                    train_countries, 
                                                    resample=resample_freq)

In [15]:
train_ts['weather']= train_weather

In [16]:
train_ts.head()

,danceability,valence,energy,weather
Date,,,,
2017-01-01,0.686661,0.613128,0.733457,13.724146
2017-01-08,0.680100,0.575376,0.717565,13.026443
2017-01-15,0.678670,0.571145,0.715792,13.737897
2017-01-22,0.679886,0.573374,0.716241,13.667816
2017-01-29,0.678062,0.572179,0.715675,13.370844


In [17]:
eval_ts = generate_target_multivariate_timeseries(ranking_features_df, 
                                                    eval_countries, 
                                                    resample=resample_freq)
eval_ts['weather']= eval_weather
eval_ts.head()

,danceability,valence,energy,weather
Date,,,,
2017-01-01,0.661483,0.516450,0.669711,4.398201
2017-01-08,0.660044,0.493551,0.663224,1.884014
2017-01-15,0.659145,0.488941,0.661017,4.045547
2017-01-22,0.658882,0.485850,0.661528,3.638521
2017-01-29,0.659581,0.487757,0.660800,4.218135


# Data visualization

In [18]:
train_ts.head().describe()

,danceability,valence,energy,weather
count,5.000000,5.000000,5.000000,5.000000
mean,0.680676,0.581040,0.719746,13.505429
std,0.003451,0.018006,0.007701,0.306508
min,0.678062,0.571145,0.715675,13.026443
25%,0.678670,0.572179,0.715792,13.370844
50%,0.679886,0.573374,0.716241,13.667816
75%,0.680100,0.575376,0.717565,13.724146
max,0.686661,0.613128,0.733457,13.737897


### Cointegration test

In [19]:
def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,7)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    
    def adjust(val, length= 6): 
        return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

    print(out.trace_stat)

In [20]:
cointegration_test(train_ts)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
danceability ::  46.6      > 40.1749   =>   True
valence ::  21.21     > 24.2761   =>   False
energy ::  7.68      > 12.3212   =>   False
weather ::  0.04      > 4.1296    =>   False
[4.66027604e+01 2.12052506e+01 7.67887585e+00 4.05244014e-02]


### Granger test

In [21]:
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(train_ts, variables = train_ts.columns)

,danceability_x,valence_x,energy_x,weather_x
danceability_y,1.0000,0.0336,0.0001,0.0258
valence_y,0.0000,1.0000,0.0000,0.0560
energy_y,0.0000,0.0037,1.0000,0.0011
weather_y,0.1352,0.0596,0.1379,1.0000


In [22]:
train_ts

,danceability,valence,energy,weather
Date,,,,
2017-01-01,0.686661,0.613128,0.733457,13.724146
2017-01-08,0.680100,0.575376,0.717565,13.026443
2017-01-15,0.678670,0.571145,0.715792,13.737897
2017-01-22,0.679886,0.573374,0.716241,13.667816
2017-01-29,0.678062,0.572179,0.715675,13.370844
...,...,...,...,...
2022-01-30,0.704686,0.587690,0.680992,14.521034
2022-02-06,0.702096,0.587309,0.681579,14.291322
2022-02-13,0.701434,0.586140,0.681857,14.490484


### Augmented Dickey-Fuller Test

In [23]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

#### Train data

In [24]:
adfuller_test(train_ts['danceability'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -3.2274
 No. Lags Chosen       = 1
 Critical value 1%     = -3.455
 Critical value 5%     = -2.872
 Critical value 10%    = -2.573
 => P-Value = 0.0185. Rejecting Null Hypothesis.
 => Series is Stationary.


In [25]:
adfuller_test(train_ts['valence'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -1.2729
 No. Lags Chosen       = 16
 Critical value 1%     = -3.456
 Critical value 5%     = -2.873
 Critical value 10%    = -2.573
 => P-Value = 0.6415. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


In [26]:
adfuller_test(train_ts['energy'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.4368
 No. Lags Chosen       = 2
 Critical value 1%     = -3.455
 Critical value 5%     = -2.872
 Critical value 10%    = -2.573
 => P-Value = 0.1316. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


#### Eval data

In [27]:
adfuller_test(eval_ts['danceability'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -4.2348
 No. Lags Chosen       = 0
 Critical value 1%     = -3.455
 Critical value 5%     = -2.872
 Critical value 10%    = -2.573
 => P-Value = 0.0006. Rejecting Null Hypothesis.
 => Series is Stationary.


In [28]:
adfuller_test(eval_ts['valence'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.4212
 No. Lags Chosen       = 2
 Critical value 1%     = -3.455
 Critical value 5%     = -2.872
 Critical value 10%    = -2.573
 => P-Value = 0.1358. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


In [29]:
adfuller_test(eval_ts['energy'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.6326
 No. Lags Chosen       = 1
 Critical value 1%     = -3.455
 Critical value 5%     = -2.872
 Critical value 10%    = -2.573
 => P-Value = 0.0864. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


We diferenciate the time series to make it stationary

In [30]:
train_ts_differenced = train_ts.diff().dropna()
diff_=True

In [31]:
adfuller_test(train_ts_differenced['danceability'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -13.8858
 No. Lags Chosen       = 1
 Critical value 1%     = -3.455
 Critical value 5%     = -2.872
 Critical value 10%    = -2.573
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


In [32]:
adfuller_test(train_ts_differenced['valence'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -13.4325
 No. Lags Chosen       = 1
 Critical value 1%     = -3.455
 Critical value 5%     = -2.872
 Critical value 10%    = -2.573
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


In [33]:
adfuller_test(train_ts_differenced['energy'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -14.9911
 No. Lags Chosen       = 1
 Critical value 1%     = -3.455
 Critical value 5%     = -2.872
 Critical value 10%    = -2.573
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


In [34]:
eval_ts_differenced = eval_ts.diff().dropna()
eval_diff_=True

In [35]:
adfuller_test(eval_ts_differenced['danceability'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -17.2907
 No. Lags Chosen       = 0
 Critical value 1%     = -3.455
 Critical value 5%     = -2.872
 Critical value 10%    = -2.573
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


In [36]:
adfuller_test(eval_ts_differenced['valence'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -12.302
 No. Lags Chosen       = 1
 Critical value 1%     = -3.455
 Critical value 5%     = -2.872
 Critical value 10%    = -2.573
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


In [37]:
adfuller_test(eval_ts_differenced['energy'])

    Augmented Dickey-Fuller Test on "" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -18.8763
 No. Lags Chosen       = 0
 Critical value 1%     = -3.455
 Critical value 5%     = -2.872
 Critical value 10%    = -2.573
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


### Select order for VAR Model

In [38]:
model = VAR(train_ts_differenced)

In [39]:
x = model.select_order(maxlags=24)
x.summary()

,AIC,BIC,FPE,HQIC
0,-35.27,-35.21,4.821e-16,-35.25
1,-35.58*,-35.30*,3.522e-16*,-35.47*
2,-35.57,-35.06,3.563e-16,-35.36
3,-35.52,-34.78,3.737e-16,-35.22
4,-35.53,-34.56,3.716e-16,-35.14
5,-35.48,-34.28,3.914e-16,-35.00
6,-35.44,-34.01,4.057e-16,-34.87
7,-35.40,-33.74,4.236e-16,-34.73
8,-35.34,-33.45,4.512e-16,-34.58
9,-35.26,-33.14,4.908e-16,-34.41


In [40]:
n_lags=2

In [41]:
model_fitted = model.fit(n_lags)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 20, Aug, 2022
Time:                     17:05:36
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                   -35.0091
Nobs:                     267.000    HQIC:                  -35.2984
Log likelihood:           3258.85    FPE:                3.85257e-16
AIC:                     -35.4927    Det(Omega_mle):     3.37411e-16
--------------------------------------------------------------------
Results for equation danceability
                     coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------
const                   0.000075         0.000217            0.346           0.729
L1.danceability        -0.100730         0.104559           -0.963           0.335
L1.valence              0.078957         0.07

In [42]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)

def adjust(val, length= 6): return str(val).ljust(length)


for col, val in zip(train_ts_differenced.columns, out):
    print(adjust(col), ':', round(val, 2))

danceability : 2.03
valence : 1.98
energy : 2.02
weather : 1.94


### Forecast test

In [43]:
def rolling_window(values_lst, window_size, n_features):
    array= []
    for i in range(values_lst.shape[0]-window_size+1):
        if array is not None:
            array.append(values_lst[i:i+window_size,:].reshape(window_size,n_features))
        else:
            array = values_lst[i:i+window_size,:].reshape(1,window_size,n_features)
    return np.array(array)

def mape_fn(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100


def compute_metrics_as_dataframe_fn(y_valid, y_hat):
    metrics= []
    metrics_global = {'mse':[],'rmse':[],'mae':[],'cvrmse':[],'mape':[],}
    for f in range(y_valid.shape[-1]):
        for t in range(y_valid.shape[1]):
            mae = mean_absolute_error(y_valid[:,t,f], y_hat[:,t,f])
            mse = mean_squared_error(y_valid[:,t,f], y_hat[:,t,f])
            rmse= mean_squared_error(y_valid[:,t,f], y_hat[:,t,f], squared = False)
            cvrmse =  rmse/np.mean(y_valid[:,t,f])*100 # it is a percentage
            mape = mape_fn(y_valid[:,t,f], y_hat[:,t,f])

            metrics.append((f,t, mae, mse, rmse, cvrmse, mape))

    features_map = {0:'danceability', 1:'valence', 2:'energy', 3:'weather'}
    metrics_df = pd.DataFrame.from_records(metrics, columns='feature T MAE MSE RMSE CVRMSE MAPE'.split())
    metrics_df['feature']= metrics_df['feature'].apply(lambda x: features_map[x])
    metrics_df = metrics_df.set_index('T')
    metrics_df.loc['global']= metrics_df.mean(axis=0)
    
    return metrics_df

In [44]:
y_hat = []
n_steps_ahead = 16
nfeatures=4
#n_lags = 8
lr= 0.8 # learning rate

training_size= int(train_ts_differenced.shape[0]* lr)
for i in range(training_size, train_ts_differenced.shape[0]-n_steps_ahead+1):
    X= train_ts_differenced.iloc[:i]
    X_no_diff= train_ts.iloc[1:i+1]
    if not diff_:
        X_no_diff= target_ts.iloc[:i]
    
    Y= eval_ts_differenced.iloc[:i]
    Y_no_diff= eval_ts.iloc[1:i+1]
    if not eval_diff_:
        Y_no_diff= eval_ts.iloc[:i]
    
    model = VAR(X)
    model_fitted = model.fit(n_lags)

    prediction_= model_fitted.forecast(y=Y.values[-n_lags:], steps=n_steps_ahead)
    
    df_forecast = pd.DataFrame(prediction_, 
                               index=eval_ts_differenced.iloc[i:i+n_steps_ahead].index, 
                               columns=eval_ts_differenced.columns + '_1d')

    if diff_:
        columns = eval_ts_differenced.columns
        for col in columns:  
            df_forecast[str(col)] = Y_no_diff[col].iloc[-1] + df_forecast[str(col)+'_1d'].cumsum()
    else:
        for col in columns:  
            df_forecast[str(col)] = df_forecast[str(col)+'_1d']

    #rint(target_ts.iloc[i:i+n_steps_ahead], df_forecast)
    y_hat_lst = df_forecast[columns].values
    y_hat.append(y_hat_lst)


y_true = train_ts.iloc[training_size+1:].values
if not diff_:
    y_true = train_ts.iloc[training_size:].values

y_hat= np.array(y_hat)

var_metrics_df = compute_metrics_as_dataframe_fn(rolling_window(y_true, n_steps_ahead, nfeatures), y_hat)

var_metrics_df

,feature,MAE,MSE,RMSE,CVRMSE,MAPE
T,,,,,,
0,danceability,0.028889,0.000848,0.029124,4.115086,4.081330
1,danceability,0.028407,0.000825,0.028723,4.059726,4.014000
2,danceability,0.027938,0.000802,0.028318,4.004620,3.949257
3,danceability,0.027450,0.000778,0.027896,3.947228,3.881880
4,danceability,0.026954,0.000754,0.027466,3.888326,3.812968
...,...,...,...,...,...,...
12,weather,5.453566,47.453789,6.888671,40.992533,31.378375
13,weather,5.560335,48.829597,6.987818,41.746359,31.984072
14,weather,5.709142,50.982189,7.140181,42.755189,32.825178


In [45]:
var_metrics_df.T

T,0,1,2,3,4,5,6,7,8,9,...,7,8,9,10,11,12,13,14,15,global
feature,danceability,danceability,danceability,danceability,danceability,danceability,danceability,danceability,danceability,danceability,...,weather,weather,weather,weather,weather,weather,weather,weather,weather,NaN
MAE,0.028889,0.028407,0.027938,0.02745,0.026954,0.026492,0.025699,0.025559,0.025371,0.025186,...,4.500334,4.711941,4.922283,5.097572,5.268925,5.453566,5.560335,5.709142,5.778668,1.224064
MSE,0.000848,0.000825,0.000802,0.000778,0.000754,0.000733,0.000707,0.000704,0.000693,0.000686,...,36.142186,38.223039,40.201249,42.373817,44.839044,47.453789,48.829597,50.982189,52.767056,9.625481
RMSE,0.029124,0.028723,0.028318,0.027896,0.027466,0.027077,0.026581,0.026537,0.026326,0.026199,...,6.011837,6.182478,6.340446,6.509517,6.696196,6.888671,6.987818,7.140181,7.264094,1.580872
CVRMSE,4.115086,4.059726,4.00462,3.947228,3.888326,3.835094,3.768492,3.762382,3.732825,3.715121,...,35.372792,36.439104,37.458011,38.564984,39.755747,40.992533,41.746359,42.755189,43.782427,15.157748
MAPE,4.08133,4.014,3.949257,3.88188,3.812968,3.748616,3.636534,3.616332,3.590106,3.563963,...,26.489178,27.521131,28.591995,29.541064,30.454831,31.378375,31.984072,32.825178,33.193776,12.926252


In [46]:
var_metrics_df[var_metrics_df['feature']=='energy'].T.loc['MAE RMSE CVRMSE'.split()][[0,1,3,7,9,11,15]]

T,0,1,3,7,9,11,15
MAE,0.043681,0.043953,0.044106,0.044524,0.045354,0.046102,0.047601
RMSE,0.044477,0.044765,0.044894,0.045998,0.047147,0.048189,0.050288
CVRMSE,6.550153,6.59164,6.611807,6.776793,6.941042,7.090007,7.389651


In [47]:
var_metrics_df[var_metrics_df['feature']=='energy'].loc[[0,1,3,7,9,11,15]].mean().round(3)

MAE       0.045
MSE       0.002
RMSE      0.047
CVRMSE    6.850
MAPE      6.630
dtype: float64

In [48]:
var_metrics_df.to_csv(os.path.join('prediction_results', 
                                   f'var_metrics_{train_clust}_{eval_clust}_{resample_freq}_diff.csv'))

In [49]:
print("That's all folks!")

That's all folks!
